# Setting up a strategy-based portfolio

In this training module we'll see how to use LUSID to perform the following task:

**<div align="center">As a portfolio manager, I want to load transactions from two systems, each with different transaction codes, into separate investment strategies in a portfolio. I want LUSID to calculate instrument holdings grouped by strategy, and understand LUSID's view of my positions on trade date vs settlement date.</div>**

In [ ]:
!pip3 install -U lusid-sdk finbourne-sdk-utils

In [ ]:
# Set up LUSID
import os
import pandas as pd
import json
import uuid
import matplotlib.pyplot as plt
from IPython.core.display import HTML
import logging
logging.basicConfig(level=logging.INFO)

import lusid as lu
import lusid.api as la
import lusid.models as lm

from lusidjam import RefreshingToken
from lusid.extensions import (
    SyncApiClientFactory,
    ArgsConfigurationLoader,
    EnvironmentVariablesConfigurationLoader,
    SecretsFileConfigurationLoader
)
from finbourne_sdk_utils.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from finbourne_sdk_utils.jupyter_tools import StopExecution
from finbourne_sdk_utils.lpt.lpt import to_date

# Set pandas display options
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.options.display.float_format = "{:,.2f}".format

# Authenticate to SDK
# Run the Notebook in Jupyterhub for your LUSID domain and authenticate automatically
secrets_path = os.getenv("FBN_SECRETS_PATH")
# Run the Notebook locally using a secrets file (see https://support.lusid.com/knowledgebase/article/KA-01663)
if secrets_path is None:
    secrets_path = os.path.join(os.path.dirname(os.getcwd()), "secrets.json")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
config_loaders=[
    ArgsConfigurationLoader(access_token = RefreshingToken(), app_name = "LusidJupyterNotebook"),
    EnvironmentVariablesConfigurationLoader(),
    SecretsFileConfigurationLoader(secrets_path)]
api_factory = SyncApiClientFactory(config_loaders=config_loaders)
    
# Confirm success by printing SDK version
api_status = pd.DataFrame(api_factory.build(lu.ApplicationMetadataApi).get_lusid_versions().to_dict())
display(api_status)

In [ ]:
# Create a scope and code to segregate data in this module from other modules
module_scope = "FBNUniversity"
module_code = "Module-3-1"
print(f"'{module_scope}\{module_code}' scope and code created.")

## 1. Examining the transaction source files
Note equity trades occur on 1 March 2022 but do not settle until 3 March.

In [ ]:
# Read transactions from SystemA into pandas dataframe
SystemA_df = pd.read_csv("data/SystemA.csv", keep_default_na = False)
display(SystemA_df)

In [ ]:
# Read transactions from SystemB into pandas dataframe
SystemB_df = pd.read_csv("data/SystemB.csv", keep_default_na = False)
display(SystemB_df)

## 2. Ensuring instruments are mastered correctly

It's possible the equity instruments in our transaction source files are already mastered in LUSID as part of the demonstration data, but for the avoidance of doubt we'll master them separately in a segregated custom instrument scope (the `GBP` currency instrument is mastered out-of-the-box, in the `default` instrument scope).

In [ ]:
# Obtain the LUSID Instruments API
instruments_api = api_factory.build(la.InstrumentsApi)

# Create a convenience function to call for each system dataframe
def create_and_upsert_instruments(system_dataframe):

    # Create a dictionary of instrument definitions
    definitions = {}

    # Iterate over each row in the system dataframe
    for index, security in system_dataframe.iterrows():

        # Model equities
        if security["asset"] == "Equity":
            # Create definitions
            definitions[security["instrument"]] = lm.InstrumentDefinition(
                name = security["instrument"],
                identifiers = {
                    "Figi": lm.InstrumentIdValue(value = security["figi"]),
                },
                definition = lm.Equity(
                    instrument_type = "Equity",
                    dom_ccy = security["currency"],
                    identifiers = {}
                )
            )

    # Upsert instruments to LUSID
    upsert_instruments_response = instruments_api.upsert_instruments(
        request_body = definitions,
        # Master the instruments in a custom scope
        scope = f"{module_scope}{module_code}",
    )

    # Transform API response to a dataframe and show internally-generated unique LUID for each mastered instrument
    upsert_instruments_response_df = lusid_response_to_data_frame(list(upsert_instruments_response.values.values()))
    display(upsert_instruments_response_df[["name", "lusidInstrumentId"]])
    
    
# Master equities from SystemA in a custom instrument scope
create_and_upsert_instruments(SystemA_df)
# Master equities from SystemB in the same custom instrument scope
create_and_upsert_instruments(SystemB_df)

## 3. Creating a portfolio

We need to create a ['sub-holding key' (SHK)](https://support.lusid.com/docs/what-is-a-sub-holding-key-shk) and register it with the portfolio. Note we must also set the instrument scope of the portfolio to be the scope in which our instruments are mastered.

### 3.1 Creating a property definition for the SHK

An SHK is a custom property and so requires a property definition in the standard way. SHKs live in the `Transaction` domain rather than `Portfolio`, since the properties are applied to transactions and not to the portfolio itself.

In [ ]:
# Obtain the LUSID Property Definition API
property_definition_api = api_factory.build(la.PropertyDefinitionsApi)

# Create a property definition for the SHK in the 'Transaction' domain, with a unique scope and code
property_definition = lm.CreatePropertyDefinitionRequest(
    domain = "Transaction",
    scope = module_scope,
    code = module_code,
    display_name = "Investment strategy",
    data_type_id = lm.ResourceId(
        scope = "system",
        code = "string"
    )
)

# Upsert property definition to LUSID
try:
    upsert_property_definition_response = property_definition_api.create_property_definition(
        create_property_definition_request = property_definition
    )
    print(f"Property definition created with the following key: {upsert_property_definition_response.key}")
except lu.ApiException as e:
    if json.loads(e.body)["name"] == "PropertyAlreadyExists":
        logging.info(
            f"Property definition with the following key already exists: {property_definition.domain}/{property_definition.scope}/{property_definition.code}"
        )

# Capture SHK 3-stage property key for future use
sub_holding_key = f"{property_definition.domain}/{property_definition.scope}/{property_definition.code}"

### 3.2 Creating the portfolio and registering the SHK
The SHK is registered using the `sub_holdings_keys` field, and the custom instrument scope using the `instrument_scopes` field.

In [ ]:
# Obtain the LUSID Transaction Portfolio API
transaction_portfolios_api = api_factory.build(la.TransactionPortfoliosApi)

# Create portfolio definition
portfolio_definition=lm.CreateTransactionPortfolioRequest(
    display_name="Training module 3.1",
    code = module_code,
    base_currency = "GBP",
    # Must be before first transaction recorded
    created="2022-01-01T00:00:00Z",
    # Register the SHK property with the portfolio
    sub_holding_keys = [sub_holding_key],
    # Attempt to resolve transactions to instruments in the custom scope before falling back to the default scope
    instrument_scopes = [f"{module_scope}{module_code}"],
)

# Upsert portfolio to LUSID, making sure it's not already there
try:
    create_portfolio_response=transaction_portfolios_api.create_portfolio(
        scope = module_scope,
        create_transaction_portfolio_request = portfolio_definition
    )
    # Confirm success
    print(f"Portfolio with display name '{create_portfolio_response.display_name}' created effective {str(create_portfolio_response.created)}")
except lu.ApiException as e:
    if json.loads(e.body)["name"] == "PortfolioWithIdAlreadyExists":
            logging.info(json.loads(e.body)["title"])

## 4. Configuring transaction types to determine economic impacts

We want to create custom `BuyEQ` and `Acheter` transaction types that, when assigned to transactions representing equity purchases, have the same economic impact as the built-in `Buy` transaction type, namely to:

1. Increase your holding in each equity instrument by the number of units bought.
2. Decrease your trade date cash position in the transaction currency by the total consideration.

In [ ]:
# Obtain the LUSID System Configuration API
transaction_config_api = api_factory.build(la.TransactionConfigurationApi)

# Create a convenience function to call for each custom transaction type
def configure_new_transaction_type(transaction_type_code):
    transaction_type_request = lm.TransactionTypeRequest(
        # Create a new alias with the transaction code as the type (most other settings are replicated from the
        # built-in Buy transaction type)
        aliases = [
            lm.TransactionTypeAlias(
                type = transaction_type_code,
                description = "The purchase of an equity",
                transaction_class = "Basic",
                transaction_roles = "LongLonger"
            )
        ],
        # Replicate the movements from the built-in Buy transaction type
        movements = [
            lm.TransactionTypeMovement(
                movement_types = "StockMovement",
                side = "Side1",
                direction = 1,
                name = "Increase the holding by the number of units"
            ),
            lm.TransactionTypeMovement(
                movement_types = "CashCommitment",
                side = "Side2",
                direction = -1,
                name = "Decrease cash position by total cost"
            )
        ]
    )
    
    # Upsert transaction type to LUSID    
    try:
        transaction_config_api.set_transaction_type(
            source = module_scope,
            type = transaction_type_code,
            transaction_type_request = transaction_type_request
        )
        print(f"Transaction type '{transaction_type_code}' created.")
    except lu.ApiException as e:
        if json.loads(e.body)["name"] == "TransactionTypeDuplication":
            logging.info(
                f"Transaction type '{transaction_type_code}' already exists."
            )

# Create a new transaction type representing the 'BuyEQ' transaction code in SystemA
configure_new_transaction_type("BuyEQ")
# Create a new transaction type representing the 'Acheter' transaction code in SystemB
configure_new_transaction_type("Acheter")

## 5. Loading transactions into investment strategies
We assign the 3-stage property key of the SHK to the `properties` field on each transaction, with the name of the strategy as the property value.

In [ ]:
# Create a convenience function to call for each system dataframe
def load_transactions_from_source_files(system_dataframe, strategy):
    
    # Create list of transactions to upsert
    transactions = []
    
    # For each row in dataframe
    for index, txn in system_dataframe.iterrows():
        
        # Set instrument identifiers based on whether or not instrument is cash
        if txn["txn_type"] == "FundsIn":
            identifiers = {"Instrument/default/Currency": txn["currency"]}
        else:
            identifiers = {"Instrument/default/Figi": txn["figi"]}    

        transactions.append(
            lm.TransactionRequest(
                transaction_id = txn["txn_id"],
                # Map the transaction code to a custom transaction type
                type = txn["txn_type"],
                instrument_identifiers = identifiers,
                transaction_date = txn["trade_date"],
                settlement_date = txn["settle_date"],
                units = txn["units"],
                transaction_price = lm.TransactionPrice(price = txn["price"], type="Price"),
                total_consideration = lm.CurrencyAndAmount(
                    # Calculate cost on-the-fly
                    amount = txn["units"] * txn["price"],
                    currency = txn["currency"]
                ),
                # Assign the SHK property to each transaction so it is loaded into an investment strategy
                properties = {
                    f"{sub_holding_key}": lm.PerpetualProperty(
                        key = f"{sub_holding_key}",
                        value = lm.PropertyValue(label_value = strategy)
                    )
                },
                # Identify the source of the custom transaction type (if omitted, uses the 'default' source)
                source = module_scope
            )
        )

    # Upsert transactions to LUSID
    upsert_transactions_response = transaction_portfolios_api.upsert_transactions(
        scope = module_scope, 
        code = module_code, 
        transaction_request = transactions
    )
    
    display(f"Transactions loaded at {str(upsert_transactions_response.version.as_at_date)}")
    
# Load transactions from SystemA into the portfolio's Growth strategy    
load_transactions_from_source_files(SystemA_df, "Growth")
# Load transactions from SystemB into the portfolio's Income strategy     
load_transactions_from_source_files(SystemB_df, "Income")

## 6. Calculating holdings grouped by strategy on the trade date

Providing we have: 

1. Registered the SHK with the portfolio
2. Applied the SHK property to each transaction

...LUSID automatically groups holdings into strategies. We can call the LUSID `GetHoldings` API with an explicit date of 1 March 2022 to understand LUSID's holding calculation on the trade date. Note `units` and `settled_units` differ for equities, and that cash lines with a `holding_type` of `C` reflect committed cash.

In [ ]:
# Get holdings for portfolio effective 1 March 2022
get_holdings_response=transaction_portfolios_api.get_holdings(
    scope = module_scope, 
    code = module_code,
    # Specify an explicit date
    effective_at = "2022-03-01",
    # Decorate the instrument name property onto holdings to make the API response more intuitive
    property_keys=["Instrument/default/Name"],
)

# Transform API response to a pandas dataframe and show it
get_holdings_response_df=lusid_response_to_data_frame(get_holdings_response, rename_properties=True)
# Drop some noisy columns
get_holdings_response_df.drop(columns=[
   "instrumentScope", "costPortfolioCcy.currency", "currency", "properties.Holding/default/SourcePortfolioId.value.labelValue", "properties.Holding/default/SourcePortfolioId.effectiveFrom", "properties.Holding/default/SourcePortfolioScope.value.labelValue", "properties.Holding/default/SourcePortfolioScope.effectiveFrom" ], inplace=True)
display(get_holdings_response_df)

## 7. Calculating holdings again post-settlement date

If we call `GetHoldings` API without an explicit date, LUSID uses today's date. Since this is after the settlement date, we can see that `units` and `settled_units` are now the same for equities, and that cash has been disbursed.

In [ ]:
# Get holdings for portfolio effective today
get_holdings_response=transaction_portfolios_api.get_holdings(
    scope = module_scope, 
    code = module_code,
    # Decorate the instrument name property onto holdings to make the API response more intuitive
    property_keys=["Instrument/default/Name"],
)

# Transform API response to a pandas dataframe and show it
get_holdings_response_df=lusid_response_to_data_frame(get_holdings_response, rename_properties=True)
# Drop some noisy columns
get_holdings_response_df.drop(columns=[
   "instrumentScope", "costPortfolioCcy.currency", "currency", "properties.Holding/default/SourcePortfolioId.value.labelValue", "properties.Holding/default/SourcePortfolioId.effectiveFrom", "properties.Holding/default/SourcePortfolioScope.value.labelValue", "properties.Holding/default/SourcePortfolioScope.effectiveFrom" ], inplace=True)
display(get_holdings_response_df)